In [1]:
import auxiliary_tools

In [2]:
df_proposals_shrt, requests_ids = auxiliary_tools.prepProposals("../proposals/")
df_requests_shrt = auxiliary_tools.prepRequests("../requests/", requests_ids)
df_rides_shrt = auxiliary_tools.prepRides("../rides/")

# proposals:  123402
# requests 121427
# rides 46616


In [ ]:
df_drt_dataset = auxiliary_tools.mergePropRequRides(df_proposals_shrt, df_requests_shrt, df_rides_shrt)

In [ ]:
df_routes, _ = auxiliary_tools.routesTransformation("../data/drt_dataset.csv")

In [ ]:
df_van_50 = df_routes[df_routes['VAN_ID'] == '50']

In [ ]:
individual = df_van_50[['LATITUDE', 'LONGITUDE']].values.tolist()

In [ ]:
import numpy as np
from tqdm import tqdm

def client_fitness(route_requests, individual):
    c_fitness = []
    print("Client Fitness")
    for i in range(len(route_requests)):
        request_r = route_requests[i]
        request_origin = [request_r[0], request_r[1]]
        print(request_origin)
        vs_individual = individual[i]
        vs_destination = vs_individual
        c_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(request_origin, vs_destination, 5001, 'walking'))
    fitness_value = np.sum([f[0] for f in c_fitness])
    return fitness_value

def operator_fitness(individual, penalty_const):
    ori_dest = [(first, second) for first, second in zip(individual, individual[1:])]
    penalty_sum = 0
    
    for pair in ori_dest:
        if (pair[0] != pair[1]) == True :
            penalty_sum+=penalty_const
    o_fitness = []
    print("Operator Fitness")
    for od_r in ori_dest:
        o_fitness.append(auxiliary_tools.getGeoDistanceETA_OSRM(od_r[0], od_r[1], 5000, 'driving'))
        
    fitness_value = np.sum([f[0] for f in o_fitness]) + penalty_sum
    return fitness_value

def fitness(individual, route_requests, penalty_const):
    import time
    start_time = time.time()
    from pexecute.thread import ThreadLoom
    loom = ThreadLoom(max_runner_cap=10)
    
    loom.add_function(client_fitness, [route_requests, individual], {})
    loom.add_function(operator_fitness, [individual, penalty_const], {})

#     client_f = client_fitness(route_requests, individual)
#     operator_f = operator_fitness(individual, penalty_const)
    output = loom.execute()
    client_f = output[0]['output']
    operator_f = output[1]['output']
    print("--- %s seconds ---" % round(time.time() - start_time))
    return client_f, operator_f

In [ ]:
while True:
    fitness(individual, individual, 0.2)